<a href="https://colab.research.google.com/github/mahwaik/tutorials/blob/master/BD_24_Summative1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SETUP

You will need to upload `gen_data.py` for this to work. You'll also need to set up spark as in the spark notebooks.

In [1]:
# Installing java and downloading spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Summative

This exercise will go through a simulated streaming data workflow, as seen in the streaming data module. But it will be more complex, and closer to what a real world scenario might look like. You will need to 
- Investigate the incoming data
- Create an appropriate database to store the incoming records
- Write code to process records one by one as they arrive, including
 - Printing warnings when any reading goes above a predefined threshold
 - Storing the incoming data in a database
- Write code to analyse the stored data in a scalable manner
- Display relevant information in a dashboard

## Create an appropriate database

Each incoming record will look something like the following: 

{'Device_ID': 9,                  
  'Temp1': 33.01235436945101,  
  'Temp2': 46.313589806396116,  
  'Temp3': 16.506177184725505,  
  'Temp_Ambient': 23.782493817278034}
  
Each device is assigned an integer ID. Every device has multiple sensors, and reports the readings from each sensor as a float. Each call to gen_data.getReading() returns a time (an integer here to make things easier) and a record that follows the same pattern as above.

<b>Create a database to store the incoming data.</b> Include a time field for the time that the data arrives. If you create the database with python, show the code here, otherwise include any bash or sql code you run. You may wish to come back and add additional fields to make later analysis easier.

### ON COLAB: You can just show the appropriate SQL code, and then continue using an alternative (like a dataframe, saved to CSV) for the rest of the assignment.

In [ ]:
CREATE DATABASE TEMPERATURE_READINGS
CREATE TABLE Gen_data(
  Device_ID varchar(255),
  Temp1 int,
  Temp2 int,
  Temp3 int,
  Temp_Ambient int,
);

## Store and process the incoming data

As each record arrives (i.e. each loop of the for loop), you must
- <b>Store the record in the database you created above
- Use either moving windows or exponential averaging to keep track each sensor value for each device. Print out the values at the end of the loop
- Print a warning if any reported temperature exceeds 100 degrees for the first time for that device</b>

In [183]:
import gen_data 

# Your code here for any initializations you may need
data=[]
for i in range(1000): # Hint: make this lower for testing
    
    # The simulated data arriving - don't change this
    arrival_time, record = gen_data.getReading()
    
    # Your code here

# Your code here
    
# Print out whatever states you kept track of here (such as moving averages or latest readings )

In [184]:

import pandas as pd

data.append(record)
df=pd.DataFrame(data, columns=['Device_ID','Temp1','Temp2','Temp3','Temp_Ambient'])
df

,Device_ID,Temp1,Temp2,Temp3,Temp_Ambient
0,10,47.677893,62.445682,223.838946,22.285291


In [185]:
Moving_av_Temp1= df.iloc[:,1].rolling(window=1).mean()
Moving_av_Temp1


0    47.677893
Name: Temp1, dtype: float64

In [186]:
Moving_av_Temp2= df.iloc[:,2].rolling(window=1).mean()
Moving_av_Temp2

0    62.445682
Name: Temp2, dtype: float64

In [187]:
Moving_av_Temp3= df.iloc[:,3].rolling(window=10).mean()
Moving_av_Temp3

0   NaN
Name: Temp3, dtype: float64

In [188]:
Moving_av_Temp_AM= df.iloc[:,4].rolling(window=2).mean()
Moving_av_Temp_AM

0   NaN
Name: Temp_Ambient, dtype: float64

In [190]:
df['Moving_av_Temp1'] = Moving_av_Temp1
df['Moving_av_Temp2'] = Moving_av_Temp2
df['Moving_av_Temp3'] = Moving_av_Temp3
df['Moving_av_Temp_AM'] = Moving_av_Temp_AM

In [191]:
import numpy as np

In [192]:
df

,Device_ID,Temp1,Temp2,Temp3,Temp_Ambient,Moving_av_Temp1,Moving_av_Temp2,Moving_av_Temp3,Moving_av_Temp_AM
0,10,47.677893,62.445682,223.838946,22.285291,47.677893,62.445682,NaN,NaN


In [198]:
Temp1_warning = np.argmax(df['Temp1'] > 100)
if Temp1_warning ==1:
        print('Warning, Temperature exceeds 100 degrees for the first time')
        print ('Device Temp 1 on index',Temp1_warning)
else:
          print('No device over 100 degress Temp 2 device')

No device over 100 degress


In [ ]:
Temp2_warning = np.argmax(df['Temp2'] > 100)
if Temp1_warning ==1:
        print('Warning, Temperature exceeds 100 degrees for the first time for the device')
        print ('Device Temp 2 on index',Temp2_warning)
else:
          print('No device over 100 degress Temp 2 device')

In [ ]:
Temp3_warning = np.argmax(df['Temp3'] > 100)
if Temp1_warning ==1:
        print('Warning, Temperature exceeds 100 degrees for the first time for the device')
        print ('Device Temp 3 on index',Temp2_warning)
else:
          print('No device over 100 degress Temp 3 device')

In [ ]:
Temp_Ambient_warning = np.argmax(df['Temp_Ambient'] > 100)
if Temp_Ambient_warning ==1:
        print('Warning, Temperature exceeds 100 degrees for the first time for the device')
        print ('Temp_Ambient on index',Temp_Ambient_warning)
else:
          print('No device over 100 degress Temp_Ambient device')

In [163]:
#Checking Warning on Temp1
Moving_av_Temp_AM=(df['Temp1'] -21.287918).apply(abs).idxmin()
if Moving_av_Temp_AM ==1:
        print('Warning, Temperature exceeds 100 degrees for the first time for the device')
        print ('Device index',Moving_av_Temp_AM)
     


In [ ]:
#Checking Warning on Temp1
Temp1_warning=(df['Temp1'] - 100).apply(abs).idxmin()
if Temp1_warning == 1:
        print('Warning, Temperature exceeds 100 degrees for the first time for the device')
        print ('Device index',Temp1_warning)

In [139]:
if Temp1_warning > 0:
        print("There is a $25 charge for luggage that heavy.")

There is a $25 charge for luggage that heavy.


In [126]:
df.Moving_av_Temp1>44

In [66]:
window = [] # The moving window
moving_av=0

## Analyzing the stored data

You now have a nice big database. <b>Load it into spark for analysis.</b>

You are told that during the time the data was being collected, devices 3 and 10 had malfunctioning sensors - their temperature3 readings are all 200+. <b>Verify this.</b> Since the engineers knew about the faulty sensors, no harm has been done, but seeing those false readings in the historical data makes you unhappy. You decide to go the extra mile and replace these readings with slightly more believable (but still false) data, to practise your new machine learning skills.

<b>Using the other devices for training, build a model to predict temperature3 given readings from the other sensors. Use the model to replace the erroneous values with the predicted ones. 
    
Do you think this is a reasonable step to take? Explain.</b> 

## Step 4: Visualization

Time to get creative. Your final task is to build up a set of visualizations that could let an engineer get a quick overview of the current status of the system. Include the current sensor readings for each device and any metrics you think would be important to display. Choose one device and show more detail - a downsampled graph showing the readings over time, perhaps.

You don't need to have your visualizations update in real time - merely show them as they would be presented at a given instant (i.e. feel free to use all the data you stored in the first section).